In [1]:
from cataclop.ml.pipeline import factories

In [2]:
program = factories.Program.factory('default')

In [3]:
program.hash

'bfa71b6c1c53eabc6715593289d45074'

In [4]:
program.predict(dataset_params = {
    'from': '2018-07-03',
    'to': '2018-12-05'
})

loading data 394dd67233ed2729d522505ac465e9ba from cache


FileNotFoundError: [Errno 2] No such file or directory: '/Users/pourquoi/workspace/cataclop/cataclop-django/var/models/default-1.7-9460608c2660d5644db24a5692ef4858/models.joblib'

In [ ]:
program.dataset.save()

In [ ]:
program.bet(N=1, max_odds=20)
bets = program.bets
bets.head()

In [ ]:
print('{} races'.format(len(bets)))
print('from {} to {}'.format(bets['date'].min(), bets['date'].max()))

In [ ]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

bb = bets[(bets['target'].str.contains('^pred_mlp_'))].groupby('id').filter(lambda r: r['num'].nunique() == 1 ).copy()
#bb = bets.groupby('id').filter(lambda r: (r['target'] == 'pred_mlp_1').sum() == 0 and (r['target'] == 'pred_knn_10_1').sum() == 1  )

def f(r):
    n1 = r[(r['target'] == 'pred_mlp_100_1')]['num']
    n2 = r[(r['target'] == 'pred_rf_100_1')]['num']
    
    if not len(n1) and len(n2):
        return True
    if not len(n2) and len(n1):
        return True
    
    if n1.max() != n2.max():
        return True
    
    return False

#bb = bets.groupby('id').filter(f)

f = 'sub_category'

for s in bb[f].value_counts().index:
    x = bb[bb[f] == s].copy()
    print("---\n{}\t{:+.2f}\t{:+.2f}\t{:+.2f}\n---".format(s, x['profit'].sum(), x['bet'].sum(), len(x)))
    for t in x['target'].value_counts().index:
        bbb = x[(x['target'] == t) & (x['odds_ref'] > 0)].copy()
        print("{:10s}: \t {:+.2f} \t {:+.0f} \t {:+.2f}".format(t, bbb.profit.sum(), bbb.profit.count(), bbb.profit.mean()) )
        
    x['stash'] = x['profit'].cumsum()

    fig, axs = plt.subplots(1,1)
    x['stash'].plot(figsize=(6, 2))
    plt.show()

In [ ]:
program.model.features